In [1]:
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.multioutput import MultiOutputRegressor

### Helper Functions

In [2]:
def loadData(file):
    data = pd.read_csv(file)
    print('Raw shape: ',data.shape)
    data['Date'] = pd.to_datetime(data.Date)
    print('Days: ',len(set(data.Date)))
    return data

In [3]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='vehicle_count', index=['Date','Hour'],
                    columns=['DOLocationID'], aggfunc=np.sum, fill_value=0)
    return table

In [4]:
def zscoreNormalizeSpatial(matrix):
    m = matrix.copy()
    for i in range(m.shape[0]):
        m[i, :] = (m[i, :] - m[i, :].mean()) / (m[i, :].std()+1e-10)
        
    return m

In [5]:
def standardize(matrix):
    m = matrix.copy()
    scaler = StandardScaler()
    scaler.fit(m)
    t = scaler.transform(m)
    return scaler, t

In [6]:
def inverse_standardize(matrix, scaler):
    t = matrix.copy()
    return scaler.inverse_transform(t)

In [7]:
def addLag(dataset, maxlag, lagColumns):
    dataset_list = [dataset]

    for l in range(1, maxlag+1):
        df = dataset.shift(l)
        df = df[lagColumns]
        df.columns = [c+'_lag_'+str(l) for c in df.columns]
        dataset_list.append(df)

    dataset = pd.concat(dataset_list, axis=1).dropna()
    return dataset

In [8]:
def get_rmse(matrix1, matrix2):
    sumSquareError = np.mean(np.power(matrix1 - matrix2,2))
    rmse = np.power(sumSquareError,0.5)
    return rmse

In [9]:
def get_weights(rawdata, zontoBorough):
    
    rawdata['Borough'] = rawdata['DOLocationID'].apply(lambda x:zontoBorough[x])
    
    borough_df = rawdata[['vehicle_count','Borough']].groupby(by='Borough').sum().reset_index()

    zone_df = rawdata[['vehicle_count','DOLocationID']].groupby(by='DOLocationID').sum().reset_index()

    zone_df['Borough'] = zone_df['DOLocationID'].apply(lambda x:zontoBorough[x])

    zone_df = pd.merge(borough_df, zone_df, on=['Borough'], how='inner')

    zone_df['zone_weight'] = zone_df.vehicle_count_y / zone_df.vehicle_count_x

    zone_df = zone_df[['Borough', 'DOLocationID', 'zone_weight']]

    return zone_df

#### Preparing Data

In [10]:
hub = 'Jfk'
tune_hyp_params = False

In [11]:
dataDir = '/home/urwa/Documents/Projects/NYU Remote/project/data/processedData/'
file = dataDir + hub + 'VehiceByHour.csv'

In [12]:
rawdata = loadData(file)

Raw shape:  (2260080, 4)
Days:  365


In [13]:
edge_data = getTimeSeries(rawdata)
edge_data = edge_data.reset_index()
edge_data.head(3)

DOLocationID,Date,Hour,1,2,3,4,5,6,7,8,...,254,255,256,257,258,259,260,261,262,263
0,2018-01-01,0,1,0,0,1,0,0,5,0,...,0,2,4,0,0,2,3,0,5,6
1,2018-01-01,1,0,0,1,1,0,0,1,0,...,0,2,2,1,1,0,2,0,0,1
2,2018-01-01,2,0,0,0,1,0,0,1,0,...,0,0,1,0,1,0,1,0,2,0


### Community Level Aggregation

In [14]:
zones = pd.read_csv('/home/urwa/Documents/Projects/NYU Remote/project/UrbanTemporalNetworks/Data/ZonetoComm.csv')
zones.head(2)

,start_id,start_community
0,1,0.0
1,2,4.2


In [15]:
zones['start_community'] = zones.start_community.astype(str)

zontoComm = dict(zip(zones.start_id.values,zones.start_community.values))

In [16]:
comm_data = rawdata.copy(deep=True)
comm_data['DOLocationID'] = comm_data['DOLocationID'].apply(lambda x:zontoComm[x])
comm_data.head(2)

,DOLocationID,Date,Hour,vehicle_count
0,0.0,2018-01-01,0,1.0
1,4.2,2018-01-01,0,0.0


In [17]:
comm_data = getTimeSeries(comm_data)
comm_data = comm_data.reset_index()
comm_data.head(2)

DOLocationID,Date,Hour,0.0,0.1,0.2,1.0,1.1,1.2,1.3,2.0,...,4.0,4.1,4.2,4.3,4.4,4.5,5.0,5.1,5.2,5.3
0,2018-01-01,0,34,10,20,40,32,17,40,3,...,0,15,47,20,7,21,1,0,0,0
1,2018-01-01,1,18,8,20,41,18,13,18,2,...,0,11,41,2,2,5,2,0,0,0


In [18]:
zone_weights = get_weights(rawdata, zontoComm)
zone_weights.head(2)

,Borough,DOLocationID,zone_weight
0,0.0,1,0.007535
1,0.0,48,0.106139


### Merge External Data Features

In [19]:
externalDataDir = "/home/urwa/Documents/Projects/NYU Remote/project/data/HongData/"
extFile = externalDataDir + hub.upper() + ".csv"

In [20]:
extDf = pd.read_csv(extFile)
print(extDf.shape)
extDf.head(2)

(8760, 46)


,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,18/1/1 0:00,6,263,174,437,1,0,0,0,0,...,18,7,12.5,-21.2,52,0,0.0,0.0,0,0
1,18/1/1 1:00,6,138,133,271,1,0,0,0,0,...,18,7,12.5,-21.2,52,0,0.0,0.0,0,0


In [21]:
extDf['date'] = pd.to_datetime(extDf['date'], yearfirst=True)
extDf.head(2)

,date,arrival,fhv,yellow,vehicle,ifmon,iftue,ifwed,ifthu,iffri,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,2018-01-01 00:00:00,6,263,174,437,1,0,0,0,0,...,18,7,12.5,-21.2,52,0,0.0,0.0,0,0
1,2018-01-01 01:00:00,6,138,133,271,1,0,0,0,0,...,18,7,12.5,-21.2,52,0,0.0,0.0,0,0


In [22]:
min(extDf.date), max(extDf.date)

(Timestamp('2018-01-01 00:00:00'), Timestamp('2018-12-31 23:00:00'))

In [23]:
extDf['Hour'] = extDf['date'].dt.hour
extDf['Dow'] = extDf['date'].dt.dayofweek
extDf['Date'] = extDf['date'].dt.date

In [24]:
extDf.columns

Index(['date', 'arrival', 'fhv', 'yellow', 'vehicle', 'ifmon', 'iftue',
       'ifwed', 'ifthu', 'iffri', 'ifsat', 'ifsun', 'if0', 'if1', 'if2', 'if3',
       'if4', 'if5', 'if6', 'if7', 'if8', 'if9', 'if10', 'if11', 'if12',
       'if13', 'if14', 'if15', 'if16', 'if17', 'if18', 'if19', 'if20', 'if21',
       'if22', 'if23', 'maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow', 'Hour', 'Dow',
       'Date'],
      dtype='object')

In [25]:
selected_columns = ['Date', 'Hour', 'Dow', 'arrival','maxtemp', 'mintemp', 'avgtemp', 'departure', 'hdd',
       'cdd', 'participation', 'newsnow', 'snowdepth', 'ifSnow']

In [26]:
extDf = extDf[selected_columns]

In [27]:
print(comm_data.shape)
print(extDf.shape)

(8760, 26)
(8760, 14)


In [28]:
comm_data['Date'] = pd.to_datetime(comm_data['Date'])
extDf['Date'] = pd.to_datetime(extDf['Date'])

In [29]:
comm_data = pd.merge(comm_data,extDf, on=['Date', 'Hour'], how='inner')
print(comm_data.shape)
comm_data['Date'] = comm_data['Date'].dt.date
comm_data.head()

(8760, 38)


,Date,Hour,0.0,0.1,0.2,1.0,1.1,1.2,1.3,2.0,...,maxtemp,mintemp,avgtemp,departure,hdd,cdd,participation,newsnow,snowdepth,ifSnow
0,2018-01-01,0,34,10,20,40,32,17,40,3,...,18,7,12.5,-21.2,52,0,0.0,0.0,0,0
1,2018-01-01,1,18,8,20,41,18,13,18,2,...,18,7,12.5,-21.2,52,0,0.0,0.0,0,0
2,2018-01-01,2,5,2,9,11,3,3,5,1,...,18,7,12.5,-21.2,52,0,0.0,0.0,0,0
3,2018-01-01,3,2,0,0,5,1,2,4,1,...,18,7,12.5,-21.2,52,0,0.0,0.0,0,0
4,2018-01-01,4,11,3,4,7,0,4,1,2,...,18,7,12.5,-21.2,52,0,0.0,0.0,0,0


### Train Test split

In [30]:
sep = int(0.75*len(comm_data))
sep

6570

In [31]:
trainData = comm_data[:sep]
testData = comm_data[sep:]

In [32]:
trainData.shape

(6570, 38)

In [33]:
testData.shape

(2190, 38)

### Lag Variables

In [34]:
trainData.columns

Index(['Date', 'Hour', '0.0', '0.1', '0.2', '1.0', '1.1', '1.2', '1.3', '2.0',
       '2.1', '2.2', '2.3', '3.0', '3.1', '3.2', '4.0', '4.1', '4.2', '4.3',
       '4.4', '4.5', '5.0', '5.1', '5.2', '5.3', 'Dow', 'arrival', 'maxtemp',
       'mintemp', 'avgtemp', 'departure', 'hdd', 'cdd', 'participation',
       'newsnow', 'snowdepth', 'ifSnow'],
      dtype='object')

In [35]:
lagColumns = ['0.0', '0.1', '0.2', '1.0', '1.1', '1.2', '1.3', '2.0',
       '2.1', '2.2', '2.3', '3.0', '3.1', '3.2', '4.0', '4.1', '4.2', '4.3',
       '4.4', '4.5', '5.0', '5.1', '5.2', '5.3', 'arrival']

DateColumns = ['Date']

targetColumns = ['0.0', '0.1', '0.2', '1.0', '1.1', '1.2', '1.3', '2.0',
       '2.1', '2.2', '2.3', '3.0', '3.1', '3.2', '4.0', '4.1', '4.2', '4.3',
       '4.4', '4.5', '5.0', '5.1', '5.2', '5.3']

In [36]:
maxlag = 12

dataset_train = addLag(trainData, maxlag, lagColumns)

dataset_train.shape

(6558, 338)

In [37]:
dataset_test = addLag(testData, maxlag, lagColumns)
dataset_test.shape

(2178, 338)

### Modelling

In [38]:
X_train = dataset_train.drop(targetColumns+DateColumns , axis = 1)
X_test = dataset_test.drop(targetColumns+DateColumns , axis = 1)
y_train = dataset_train[targetColumns]
y_test = dataset_test[targetColumns]

In [39]:
X_train.shape, X_test.shape

((6558, 313), (2178, 313))

In [40]:
y_train.shape, y_test.shape

((6558, 24), (2178, 24))

### Hyperparameter Tuning

In [41]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 150, stop = 300, num = 3)]
# Number of features to consider at every split
max_features = ['sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(50, 110, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2,3,4]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2,3]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [150, 225, 300], 'max_features': ['sqrt'], 'max_depth': [50, 65, 80, 95, 110, None], 'min_samples_split': [2, 3, 4], 'min_samples_leaf': [2, 3], 'bootstrap': [True, False]}


In [42]:
if tune_hyp_params:
    rf = RandomForestRegressor()
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 5, \
                                   cv = 5, verbose=2, random_state=42, n_jobs = -1)
    rf_random.fit(X_train, y_train)
    rf_random.best_params_

### Training the Best Model

In [43]:
rf2 = RandomForestRegressor(random_state = 2019, n_estimators=150, 
                           min_samples_split=3,
                           min_samples_leaf= 2, 
                           max_features= 'sqrt',
                           max_depth= None, 
                           bootstrap= False)

In [44]:
rf2.fit(X_train,y_train)

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
           max_features='sqrt', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=2, min_samples_split=3,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=None,
           oob_score=False, random_state=2019, verbose=0, warm_start=False)

In [45]:
rf2.score(X_train,y_train)

0.9852417758274324

In [46]:
rf2.score(X_test,y_test)

0.8083028005353

### Predict

In [47]:
comm_prediction = rf2.predict(X_test)
comm_prediction.shape

(2178, 24)

### Evaluate

In [48]:
get_rmse(y_test, comm_prediction)

0.0    35.838631
0.1     7.711167
0.2    23.677280
1.0    10.349319
1.1    13.387687
1.2     6.637908
1.3    15.051811
2.0     2.378005
2.1     2.770318
2.2     4.239403
2.3     2.163163
3.0     9.762840
3.1     3.379836
3.2    10.280509
4.0     0.085642
4.1     6.037661
4.2    11.834550
4.3     4.796887
4.4     1.959119
4.5     7.351964
5.0     0.735822
5.1     0.745277
5.2     0.653734
5.3     0.577998
dtype: float64

In [49]:
r2_score(y_test, comm_prediction, multioutput='variance_weighted')

0.8083028005353

In [50]:
y_test.mean()

0.0    142.108815
0.1     23.471534
0.2     80.759871
1.0     37.361341
1.1     41.770891
1.2     18.734160
1.3     47.146924
2.0      4.049128
2.1      5.617539
2.2     10.584022
2.3      3.591827
3.0     30.930211
3.1      8.267218
3.2     32.519284
4.0      0.006428
4.1     19.113407
4.2     50.767218
4.3     14.177686
4.4      3.095960
4.5     22.973829
5.0      0.522957
5.1      0.511938
5.2      0.401745
5.3      0.317723
dtype: float64

### Edge Level Evaluation

In [51]:
comm_prediction.shape

(2178, 24)

In [52]:
edge_prediction_df = pd.DataFrame(comm_prediction)
edge_prediction_df.columns = y_test.columns
edge_prediction_df.head(2)

,0.0,0.1,0.2,1.0,1.1,1.2,1.3,2.0,2.1,2.2,...,4.0,4.1,4.2,4.3,4.4,4.5,5.0,5.1,5.2,5.3
0,127.393333,23.434444,80.651111,37.884444,43.474444,23.553333,51.141111,4.033333,5.647778,13.122222,...,0.010000,24.161111,38.405556,16.594444,3.443333,25.443333,0.612222,0.574444,0.471111,0.370000
1,120.007778,20.347778,62.877778,29.596667,31.220000,15.985556,38.360000,4.036667,5.085556,9.498889,...,0.006667,19.174444,36.101111,14.102222,2.456667,19.411111,0.527778,0.481111,0.326667,0.184444


In [53]:
boroughs = list(edge_prediction_df.columns)
for bor in boroughs:
    print(bor)
    
    weight_df = zone_weights[zone_weights.Borough == bor]
    
    print(len(weight_df.DOLocationID))
    
    for b_zone,z_weight in zip(weight_df.DOLocationID.values,weight_df.zone_weight.values):        
        edge_prediction_df[b_zone] = edge_prediction_df[bor] * z_weight

0.0
15
0.1
4
0.2
21
1.0
15
1.1
17
1.2
20
1.3
10
2.0
14
2.1
19
2.2
8
2.3
10
3.0
6
3.1
4
3.2
8
4.0
1
4.1
18
4.2
22
4.3
12
4.4
5
4.5
9
5.0
4
5.1
9
5.2
4
5.3
3


In [54]:
select_cols = [c for c in edge_prediction_df.columns if c not in boroughs]
edge_prediction_df = edge_prediction_df[select_cols]
edge_prediction_df.shape

(2178, 258)

In [55]:
edge_testData = edge_data[sep+maxlag:]
select_cols = [c for c in edge_testData.columns if c not in ['Date','Hour']]
edge_testData = edge_testData[select_cols]
edge_testData.shape

(2178, 258)

In [56]:
edge_prediction_df = edge_prediction_df[edge_testData.columns]
edge_prediction_df.head(2)

,1,2,3,4,5,6,7,8,9,10,...,254,255,256,257,258,259,260,261,262,263
0,0.959869,0.002029,0.325054,2.024149,0.045260,0.121166,5.716208,0.029252,0.760783,5.099426,...,0.421437,9.088301,5.492243,1.404572,1.382620,0.468002,1.529327,3.924331,4.594280,6.114051
1,0.904221,0.001907,0.292696,1.578081,0.037906,0.060401,4.360983,0.022317,0.603763,4.793446,...,0.379484,6.816966,4.119630,1.008655,1.299659,0.421414,1.299646,3.059514,3.283948,4.370266


In [57]:
edge_testData.head(2)

DOLocationID,1,2,3,4,5,6,7,8,9,10,...,254,255,256,257,258,259,260,261,262,263
6582,1,0,0,2,0,0,4,0,1,3,...,0,7,7,2,2,0,2,3,4,5
6583,1,0,0,3,0,0,1,0,1,1,...,1,4,2,0,0,0,1,2,1,2


In [58]:
get_rmse(edge_testData.values, edge_prediction_df.values)

1.9563489326708643

In [59]:
r2_score(edge_testData.values, edge_prediction_df.values, multioutput='variance_weighted')

0.5508535606362045